In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning
import sys
import numpy as np
from collections import namedtuple
from tqdm import tqdm
import warnings

In [ ]:
cols = range(1,47)
data = pd.read_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\sample_regresion_NEW.csv', usecols=cols)
data = data.drop(['OBRISANO_NE', 'OBAVEZA_AKONTACIJE_NE', 'STORNO_NE', 'DATUM_STORNA', 'DANI_DATUM_STORNA'], axis=1)
data = data.sample(n=5000, random_state=1)

In [ ]:
dataset = data.copy()
X = dataset.drop(columns=["STORNO_DA"])
y = dataset.STORNO_DA

In [ ]:
score_data = []

In [ ]:
class result:
    iteration = int(0)
    name = ""
    complexity = int(0)
    score = 0.

In [ ]:
def validate(clf, X, y, r):
    scores = cross_val_score(clf, X, y, cv=10)
    r.score, r.score_std = np.mean(scores), np.std(scores)

In [ ]:
for random_state in range(1):
    for i in tqdm(range(1, 40+1)):
        r = result()
        r.iteration = random_state
        r.name = "decision_tree"
        clf = tree.DecisionTreeClassifier(max_depth=i+1, random_state=random_state)
        fit = clf.fit(X, y)
        r.complexity = i
        validate(clf, X, y, r)
        score_data.append(r)
        
        clf = XGBClassifier(max_depth=i+1, seed=random_state)
        r = result()
        r.iteration = random_state
        r.name = "xgboost"
        r.complexity = i
        validate(clf, X, y, r)
        score_data.append(r)

        clf = LogisticRegression(random_state=random_state)
        r = result()
        r.iteration = random_state
        r.name = "logistic-regression"
        r.complexity = i
        validate(clf, X, y, r)
        score_data.append(r)
        
        clf = GradientBoostingClassifier(n_estimators=i, random_state=random_state)
        r = result()
        r.iteration = random_state
        r.name = "gradient_boosting"
        r.complexity = i
        validate(clf, X, y, r)
        score_data.append(r)

        clf = LinearSVC(C=1, random_state=random_state)
        r = result()
        r.iteration = random_state
        r.name = "SVC-linear"
        r.complexity = i
        validate(clf, X, y, r)
        score_data.append(r)
        
        clf = SVC(C=i, kernel="rbf", gamma='auto', random_state=random_state)
        r = result()
        r.iteration = random_state
        r.name = "SVC-rbf"
        r.complexity = i
        validate(clf, X, y, r)
        score_data.append(r)


In [ ]:
df_scores = pd.DataFrame([vars(x) for x in score_data])
df_scores.to_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\models_compare.csv', index=False)

In [ ]:
#ispis srednjih vrijednosti i standardne devijacke za svaki model
mean = df_scores['score'][df_scores['name']=='decision_tree'].mean()
std = df_scores['score_std'][df_scores['name']=='decision_tree'].mean()
print("decision_tree  --> mean: "+str(mean)+";  std: "+str(std))

In [ ]:
mean = df_scores['score'][df_scores['name']=='xgboost'].mean()
std = df_scores['score_std'][df_scores['name']=='xgboost'].mean()
print("xgboost  --> mean: "+str(mean)+";  std: "+str(std))

In [ ]:
mean = df_scores['score'][df_scores['name']=='gradient_boosting'].mean()
std = df_scores['score_std'][df_scores['name']=='gradient_boosting'].mean()
print("gradient_boosting  --> mean: "+str(mean)+";  std: "+str(std))

In [ ]:
mean = df_scores['score'][df_scores['name']=='SVC-linear'].mean()
std = df_scores['score_std'][df_scores['name']=='SVC-linear'].mean()
print("SVC-linear  --> mean: "+str(mean)+";  std: "+str(std))

In [ ]:
mean = df_scores['score'][df_scores['name']=='SVC-rbf'].mean()
std = df_scores['score_std'][df_scores['name']=='SVC-rbf'].mean()
print("SVC-rbf  --> mean: "+str(mean)+";  std: "+str(std))

In [ ]:
mean = df_scores['score'][df_scores['name']=='logistic-regression'].mean()
std = df_scores['score_std'][df_scores['name']=='logistic-regression'].mean()
print("logistic-regression  --> mean: "+str(mean)+";  std: "+str(std))